#### This script was created to be the only script used to prepare the ML DB for model training and evaluation and to standardize the data preparation for each of the investigations so that one process can be described and the investigations have common data preparation practices


## Dependencies and Utilities


In [51]:
import numpy as np
import os
import pandas as pd
import glob
import datetime
from os.path import isfile, join
from sys import getsizeof

def outlier_detect_and_replace(df,str_id,outlier_value):
    """This function takes a dataframe, finds any columns 
    matching 'str_id' in the provided dataframe, and replaces
    all values that exceed the 'outlier_value' with NaN"""
#     print('working on {} outlier detection and removal'.format(str_id))
#     print('outlier value = {}'.format(outlier_value))

    cols = [ col for col in df.columns if str_id in col]
#     print('will detect and replace for columns = {}'.format(cols))
    for i in cols:
        df[i][ df[i] > outlier_value ] = np.NaN

    return df



In [8]:
DMSP_FIGURE_DIR = '/Users/ryanmcgranaghan/Documents/DMSPdata/figures/'
DMSP_DATA_DIR = '/Users/ryanmcgranaghan/Documents/DMSPdata/data/'


## Prepare data for ML exploration


In [52]:
def ml_db_preparation(cols_to_drop, file_save_df_cumulative):
# def ml_db_preparation(cols_to_drop, file_save_X, file_save_y):
    ''' 
        INPUTS: 
        - cols_to_drop: list of columns in ML_DB csv files that need to be dropped from the DB that will be output
            --> Sample call to create: 
                      cols_to_drop = [c for c in df.columns if ('sin' in c) | ('cos' in c) | ('STD' in c) | ('AVG' in c)]
        - file_save_df_cumulative: filename in which to save cumulative data frame
            --> 'ML_DB_subsamp_ext_full_dfCumulative_simpleHemisphereCombine_colsDropped.csv'
        (not used currently) - file_save_X: filename in which to save X data
            --> 'ML_DB_subsamp_ext_full_X_simpleHemisphereCombine_colsDropped.csv'
        (not used currently) - file_save_y: filename in which to save y data
            --> 'ML_DB_subsamp_ext_full_y_simpleHemisphereCombine_colsDropped.csv'
    
        Dependencies:
          - DMSP_DATA_DIR: Directory location that contains the ML_DB csv files
        
    '''
    
    DMSP_DATA_DIR = '/Users/ryanmcgranaghan/Documents/DMSPdata/data/'

    flag_cumulative = 'start'

    for yr in np.arange(1987,2015):
        flag = 'start'

        print('year = {}'.format(yr))

        # CHECK FOR EXISTENCE OF DATA FOR THIS YEAR
        files = glob.glob(DMSP_DATA_DIR + 'ML_DB_subsamp_ext_'+str(yr)+'_sat*.csv')
        files.sort()

        if not files:
            print('\n\n no files for year = {} continuing...\n\n'.format(yr))
            continue

        # READ IN AND CONCATENATE ALL DATA FOR CURRENT YEAR

        for s in range(len(files)):
            sat = files[s][-7:-4]

            print('    satellite = {}'.format(sat))

    #         if (yr == 2010) & (sat == 'f16'):
    #             df_val = pd.read_csv(files[s])
    #             col_idx = df_val.columns.get_loc('SC_AACGM_LAT')
    #             df_val.loc[df_val['SC_AACGM_LAT']<=-45.,'SC_AACGM_LAT'] = df_val.loc[df_val['SC_AACGM_LAT']<=-45.,'SC_AACGM_LAT'] * -1
    #             df_val[df_val['SC_AACGM_LAT']<=45.] = np.nan

    #             # Clean up the columns and redefine the datetime as the index
    #             df_val.index = pd.DatetimeIndex( df_val['Unnamed: 0'] )
    #             df_val.index.names = ['Datetimes']
    #             df_val.drop(['Unnamed: 0','key_0'], axis=1, inplace=True)
    #             print('\n\n\n\nUsing as validation year = {} and sat = {}\n\n\n\n'.format(yr,sat))
    #             continue

            df_loop = pd.read_csv(files[s])
            # converting southern hemisphere data to positive values
            col_idx = df_loop.columns.get_loc('SC_AACGM_LAT')
            
            # Process to combine northern and southern hemisphere data
               # (a) simple approach
#             df_loop.loc[df_loop['SC_AACGM_LAT']<=-45.,'SC_AACGM_LAT'] = df_loop.loc[df_loop['SC_AACGM_LAT']<=-45.,'SC_AACGM_LAT'] * -1
               # (b) robust approach...forthcoming
            cols_By = [ col for col in df_loop.columns if 'By' in col]
            df_loop.loc[df_loop['SC_AACGM_LAT']<0.,cols_By] = df_loop.loc[df_loop['SC_AACGM_LAT']<0.,cols_By] * -1.
            df_loop.loc[df_loop['SC_AACGM_LAT']<=0.,'SC_AACGM_LAT'] = df_loop.loc[df_loop['SC_AACGM_LAT']<0.,'SC_AACGM_LAT'] * -1.

            df_loop[df_loop['SC_AACGM_LAT']<=45.] = np.nan

            # Clean up the columns and redefine the datetime as the index
            df_loop.index = pd.DatetimeIndex( df_loop['Unnamed: 0'] )
            df_loop.index.names = ['Datetimes']
            df_loop.drop(['Unnamed: 0','key_0'], axis=1, inplace=True)

            # Concatenate the full dataframe
            if flag == 'start':
                print('creating master df')
                df = df_loop
                flag = 'initiated'
            else:
                df = pd.concat([df,df_loop])
                print('size of master df = {}'.format(df.shape))

            print('        size of master df = {:.3F} MB'.format(getsizeof(df) / 10**6))




        # PREPARE DATA FOR ML

        # Remove outliers in the DB
        df = df.copy(deep=True)

        # IMF_outlier = 1000. #9999.99
        IMF_outlier = 200. #9999.99
        str_id = 'B'
        df = outlier_detect_and_replace(df,str_id,IMF_outlier)

        print('--------------------------------------------------------------')

        # vsw_outlier = 10000. #99999.9
        vsw_outlier = 2000. #99999.9
        str_id = 'vsw'
        df = outlier_detect_and_replace(df,str_id,vsw_outlier)

        print('--------------------------------------------------------------')

        # vsw_outlier = 10000. #99999.9
        vsw_outlier = 2000. #99999.9
        str_id = 'vx'
        df = outlier_detect_and_replace(df,str_id,vsw_outlier)

        print('--------------------------------------------------------------')

        # borovsky_outlier = 1e6 #df['borovsky'].quantile(.90)
        borovsky_outlier = 1e5 #df['borovsky'].quantile(.90)
        str_id = 'borovsky'
        df = outlier_detect_and_replace(df,str_id,borovsky_outlier)

        print('--------------------------------------------------------------')

        # newell_outlier = 1e5 #df['newell'].quantile(.90)
        newell_outlier = 9e3 #df['newell'].quantile(.90)
        str_id = 'newell'
        df = outlier_detect_and_replace(df,str_id,newell_outlier)

        df = df.dropna()

    #     save_filename = 'box_summary_DMSP_'+str(yr)+'.png'
    #     plot_box_summary(df,DMSP_FIGURE_DIR,save_filename) 


        # Drop unwanted features
        target_var = 'ELE_TOTAL_ENERGY_FLUX'
        print('manually overiding user-specified column dropping for time being...storing all columms')
#         df = df.drop(cols_to_drop,axis=1)


        idx = np.argwhere( df['ELE_TOTAL_ENERGY_FLUX'].to_numpy()>np.quantile(df['ELE_TOTAL_ENERGY_FLUX'].to_numpy(),.999999) )
        df.iloc[idx,:] = np.nan
        df = df.dropna()

        if flag_cumulative == 'start':
            df_cumulative = df.copy(deep=True)
            print('length of current years data = {}\nlength of cumulative training data = {}'.format(len(df),len(df_cumulative)))
            flag_cumulative = 'add'
        else:
            df_cumulative = pd.concat([df_cumulative,df])
            print('length of current years data = {}\nlength of cumulative training data = {}'.format(len(df),len(df_cumulative)))

      
    df_cumulative.to_csv(os.path.join(DMSP_DATA_DIR,file_save_df_cumulative))
    print('cumulative DF saved to {}'.format(os.path.join(DMSP_DATA_DIR,file_save_df_cumulative)))

#     X = df_cumulative[feature_cols].copy(deep=True)
#     y = df_cumulative['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
#     y[y == 0] = 0.0001
#     y = np.log10(y)
    
#     X.to_csv(os.path.join(DMSP_DATA_DIR,file_save_X))
#     print('X data saved to {}'.format(os.path.join(DMSP_DATA_DIR,file_save_X)))
    
#     y.to_csv(os.path.join(DMSP_DATA_DIR,file_save_y))
#     print('y data saved to {}'.format(os.path.join(DMSP_DATA_DIR,file_save_y)))


    
#     # Separate training and testing data
#     mask_val = [(df_cumulative.index.year == 2010) & (df_cumulative['ID_SC'].values==16)]
#     df_val = df_cumulative[mask_val[0]].copy(deep=True)
#     df_train = df_cumulative.copy(deep=True).drop( df_cumulative.index[mask_val[0]])
#     print('validation data shape = {}'.format(df_val.shape))
#     print('train data shape = {}'.format(df_train.shape))
#     print('NOTE: we will use CV on the train data below to define model training and testing data,\n  so have called the withheld data *validation* data here')

#     # Construct X and y
#     feature_cols = [c for c in df_cumulative.columns if not 'ELE' in c]


#     X_val = df_val[feature_cols].copy(deep=True)
#     y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
#     X_train = df_train[feature_cols].copy(deep=True)
#     y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
#     scaler_X = preprocessing.RobustScaler()
#     scaler_X = scaler_X.fit(X_train.values)
#     X_val_scaled = scaler_X.transform(X_val.values)
#     X_train_scaled = scaler_X.transform(X_train.values)

#     numFeatures = len(X_train.columns.to_list())
#     feature_labels = X_train.columns.to_list()

    return 
        


In [53]:
# Run the function once to create one cumulative DF
file_save_df_cumulative = 'ML_DB_subsamp_ext_full_dfCumulative_complexHemisphereCombine.csv'
ml_db_preparation([], file_save_df_cumulative)


year = 1987
    satellite = f06
creating master df
        size of master df = 647.539 MB
    satellite = f07
size of master df = (1051200, 153)
        size of master df = 1295.078 MB
    satellite = f08
size of master df = (1576800, 153)
        size of master df = 1942.618 MB
--------------------------------------------------------------
--------------------------------------------------------------
--------------------------------------------------------------
--------------------------------------------------------------
manually overiding user-specified column dropping for time being...storing all columms
length of current years data = 1884
length of cumulative training data = 1884
year = 1988
    satellite = f09
creating master df
        size of master df = 649.313 MB
--------------------------------------------------------------
--------------------------------------------------------------
--------------------------------------------------------------
------------------------

manually overiding user-specified column dropping for time being...storing all columms
length of current years data = 3275
length of cumulative training data = 1116384
year = 2010
    satellite = f16
creating master df
        size of master df = 647.539 MB
    satellite = f17
size of master df = (1051200, 153)
        size of master df = 1295.078 MB
    satellite = f18
size of master df = (1576800, 153)
        size of master df = 1942.618 MB
--------------------------------------------------------------
--------------------------------------------------------------
--------------------------------------------------------------
--------------------------------------------------------------
manually overiding user-specified column dropping for time being...storing all columms
length of current years data = 206252
length of cumulative training data = 1322636
year = 2011
    satellite = f16
creating master df
        size of master df = 647.539 MB
    satellite = f17
size of master df = 

## Save ML DBs for various explorations - note that we moved these to the functions in which they are needed

*for instance: we create the final database for hyperparameter tuning in 'Explore_ML_DB_hyperParameterExploration.ipynb'

In [54]:
# For hyperparameter tuning



In [3]:
# For feature importance



In [2]:
# For time history importance



In [ ]:
# For data volume exploration

